In [1]:
import pandas as pd
import numpy as np
from montecarlo.montecarlo import Die, Game

sides = np.array([1,2,3,5])
d = Die(sides)

/home/hrn4ch/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
type(sides)

numpy.ndarray

In [3]:
isinstance(sides, np.ndarray)

True

In [4]:
df = pd.DataFrame({'sides':sides, 'weights':[1]*len(sides)})
df = df.set_index('sides')

In [5]:
df.sample(3, replace=True, weights=df['weights']).index.to_list()

[1, 2, 1]

In [7]:
df['weights'][5]

1

In [8]:
d.change_weight(5, 4)
d.state()

,weights
sides,
1,1
2,1
3,1
5,4


In [19]:
x = np.array(d.roll(100))
np.unique(x, return_counts=True)

(array([1, 2, 3, 5]), array([12, 15, 13, 60]))

In [14]:
df.sample(5, replace=True, weights=df['weights'])

,weights
sides,
1,1
2,1
3,1
5,1
2,1


In [12]:
x = pd.DataFrame()


In [4]:
d1 = Die(np.array([1,2,3]))
d2 = Die(np.array([4,5,6]))
d3 = Die(np.array([7,8,9]))

dice = [d1,d2,d3]
g = Game(dice)

In [9]:
g.play(4)

In [10]:
g.last_play()

,0,1,2
0,2,5,7
1,2,6,9
2,2,5,8
3,1,5,8
